<a href="https://colab.research.google.com/github/dharesan/db-assignment-databros/blob/main/assignment_toy_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 11 Spark

Author: ISTD, SUTD

Title: Lab 11, Spark part 1

Date: March 5, 2025

## Learning outcome


By the end of this lesson, you are able to

* Submit PySpark jobs to a Spark cluster
* Paralelize data processing using PySpark


You can either execute this lab directly on the aws cluster with HDFS file system, or you can install PySpark in Google Colab and load the files locally. The main difference in coding is that we do not load the context from the HDFS filesystem, but instead just load a local file. Other than than that, all PySpark commands are the same.

To run this lab, you can make a copy of this notebook or `File -> Open in Playground Mode`.

## Installing PySpark in Google Colab

To install PySpark in Google Collab, execute the below cell. This will download Spark and install all necessary libraries for this lab.

In [7]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
35 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

## Wordcount Example

Let us first download the necessary data file. We can find it at `https://raw.githubusercontent.com/istd50043-2023-spring/cohort_problems/main/cc11/ex1/data.csv`.

Colab lets us execute unix commands, as long as we prepend them with `!`. So let's download the file and move it into a new folder called `input`. While we are at it, let's create a folder called `output` as well.

In [ ]:
!wget https://raw.githubusercontent.com/sutd50043/cohortclass/main/cc10/data/TheCompleteSherlockHolmes.txt
!mkdir input
!mv TheCompleteSherlockHolmes.txt input/
!mkdir output

In [ ]:
## testing commit message

You can check that the data.csv file downloaded by uncollapsing the left panel and checking the folder contents.

Now we are ready to write our PySpark code. The goal is to write a simple wordcounter:

In [ ]:
import sys
from pyspark import SparkContext, SparkConf

# sc.stop() # uncomment this during debugging to restart your context in case execution stopped mid-way this cell.

conf = SparkConf().setAppName("Wordcount Application")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

# note that we load the text file directly with a local path instead of providing an hdfs url
input_file_name = 'input/TheCompleteSherlockHolmes.txt'
text_file = sc.textFile(input_file_name)

counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

output_folder = './output/wordcount'
counts.saveAsTextFile(output_folder)

sc.stop()

In [ ]:
# your answer

## Queston 2.py


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType

spark = SparkSession.builder.appName("Assignment 2 Question 1").getOrCreate()

input_path = "/content/TA_restaurants_curated_cleaned.csv"

sc = spark.sparkContext

rdd = sc.textFile(input_path)

#rdd.collect()

header = rdd.first()
header_cols = header.split(",")
price_range_idx = header_cols.index("Price Range")
city_idx = header_cols.index("City")

print(price_range_idx,city_idx )

rdd = rdd.filter(lambda x: x[price_range_idx] != "")

rdd = rdd.map(lambda x: (x[city_idx],x[price_range_idx]))

# rdd = rdd.groupBy(lambda x: x[price_range_idx])


# for row in rdd.collect():
#     print(row)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/content/TA_restaurants_curated_cleaned.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.Spark